In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from google.colab import drive
from tensorflow import keras
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install import_ipynb

In [ ]:
import import_ipynb
import drive.MyDrive.Petrocchi.util.preprocessing as preprocessing
import drive.MyDrive.Petrocchi.util.visualization as visual

# Class dictionary

- Original class labels have been assigned to the patches according to the following mapping:
  - 0: Baseline patch
  - 1: Mass, benign
  - 2: Mass, malignant
  - 3: Calcification, benign
  - 4: Calcification, malignant

- **For contest prediction, stick to the following mapping**:
  - 0: Benign
  - 1: Malignant

# How to submit a prediction



Adapt the following snippet to your program

In [ ]:
BASE_PATH = "drive/MyDrive/Petrocchi"
MODELS_PATH = BASE_PATH + "/models"
DATA_PATH = BASE_PATH + "/data/"

In [ ]:
test_images_private = np.load(DATA_PATH + 'private_test_tensor.npy')

tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.Session()
with sess.as_default():
  print(test_images_private.shape)
  test_images_private_pp = tf.image.grayscale_to_rgb(tf.convert_to_tensor(preprocessing.preprocess_normalize_images(test_images_private, test_images_private.mean(), test_images_private.std()))).eval()
  print(test_images_private_pp.shape)

(672, 150, 150)
(672, 150, 150, 3)


In [ ]:
x_test_baseline = test_images_private_pp[0::2] 
x_test_abno = test_images_private_pp[1::2] 

test_input = [x_test_abno,x_test_abno,x_test_abno,x_test_abno,x_test_baseline,x_test_abno,x_test_baseline]
print(x_test_baseline.shape)
print(x_test_abno.shape)

(336, 150, 150, 3)
(336, 150, 150, 3)


In [ ]:
model = keras.models.load_model(os.path.join(MODELS_PATH,'ensemle_bm.h5'))

In [ ]:
# !!!Please use GPU to predict!!!
prediction = model.predict(test_input) # include baseline if you use it
output_labels = prediction.round().astype(int).reshape(-1)
assert len(output_labels) == 336 # must be of this size!
with open(BASE_PATH + '/Submission.csv','w') as mf:
  mf.writelines("\n".join(str(i) for i in output_labels) )

# How do we evaluate a submission


Read the submission

In [ ]:
path = BASE_PATH
y_pred = list(pd.read_csv(os.path.join(path,"Submission.csv"),header = None)[0].values)
assert len(y_pred) == 336 # must be of this size!


Read the true labels

In [ ]:
labels = np.load(os.path.join(DATA_PATH + 'private_test_labels.npy'))
dict_map = {1:0,3:0,2:1,4:1}
y_true = np.asarray([dict_map[x] for x in labels[labels!=0]])

In [ ]:
accuracy_score(y_true,y_pred)

In [ ]:
test_loss, test_acc = model.evaluate(test_input, y_true)
print(f"Accuracy on test set: {round(test_acc*100, 2)}% \nLoss on test set: {round(test_loss,4)}")

ensemle_bm_test_predictions = model.predict(test_input)
binary_ensemle_bm_test_predictions = ensemle_bm_test_predictions.round()
visual.plot_confusion_matrix(y_true, binary_ensemle_bm_test_predictions, ["BENIGN", "MALIGNANT"])